# 🚀 Master Notebook – Pipeline Spectroscopie DR5

Ce notebook orchestre le **workflow complet** pour entraîner un classifieur à partir des spectres LAMOST DR5 :  
**sélection d’un lot → génération/enrichissement du catalogue → prétraitement + features → entraînement → journaux & artefacts.**

**Sommaire rapide**
- [🧪 Étape 0 : SETUP & IMPORTS](#etape-0)
- [▶️ Lancer une session complète](#run-full)
- [1) Téléchargement des spectres](#step-1-download)
- [2) Sélection du lot de spectres](#step-2-select)
- [3) Catalogue : génération & enrichissement Gaia](#step-3-catalog)
- [3bis) Traitement & extraction des features](#step-3bis-features)
- [4) Entraînement du modèle](#step-4-train)

> Orchestrateur utilisé : **`MasterPipeline`**  
> Méthodes clés : `select_batch`, `generate_and_enrich_catalog`, `process_data`, `run_training_session`, `run_full_pipeline`, `interactive_training_runner`.

<a id="etape-0"></a>

#

## 🧪 Étape 0 : SETUP & IMPORTS

Initialise l’environnement, crée/valide l’arborescence des répertoires
(`RAW_DATA_DIR`, `CATALOG_DIR`, `PROCESSED_DIR`, `MODELS_DIR`, `REPORTS_DIR`)
et instancie **`MasterPipeline`**.

**Attendu après exécution :**
- un objet `pipeline` prêt à l’emploi,
- messages sur la racine du projet et, si configuré, tentative de connexion à **Gaia**.

> ℹ️ **UI interactive d’entraînement** : disponible avec `pipeline.interactive_training_runner()`.

In [1]:
from utils import setup_project_env, load_env_vars
from pipeline.master import MasterPipeline
from astroquery.gaia import Gaia
from pipeline.classifier import SpectralClassifier

# Initialisation automatique de l'environnement et des chemins
paths = setup_project_env()

# Chargement des credentials Gaia depuis .env
env_vars = load_env_vars()

try:
    print("Tentative de connexion à l'archive Gaia...")
    Gaia.login(user=env_vars.get("GAIA_USER"), password=env_vars.get("GAIA_PASS"))
    print("Connexion à Gaia réussie.")
except Exception as e:
    print(f"AVERTISSEMENT : Échec de la connexion à Gaia ({e}). Le mode 'bulk' pourrait échouer.")

# Instanciation du pipeline maître
pipeline = MasterPipeline(
    raw_data_dir=paths["RAW_DATA_DIR"],
    catalog_dir=paths["CATALOG_DIR"],
    processed_dir=paths["PROCESSED_DIR"],
    models_dir=paths["MODELS_DIR"],
    reports_dir=paths["REPORTS_DIR"],
)

print("\nSetup terminé. Tu es prêt à lancer ton pipeline.")

[INFO] Racine du projet détectée : C:\Users\alexb\Documents\Google_Cloud\alex_labs_google_sprint\astro_spectro_git
[INFO] Dossier 'src' ajouté au sys.path.
[INFO] Variables d'environnement chargées depuis C:\Users\alexb\Documents\Google_Cloud\alex_labs_google_sprint\astro_spectro_git\.env
Tentative de connexion à l'archive Gaia...
INFO: Login to gaia TAP server [astroquery.gaia.core]
INFO: OK [astroquery.utils.tap.core]
INFO: Login to gaia data server [astroquery.gaia.core]
INFO: OK [astroquery.utils.tap.core]
Connexion à Gaia réussie.

Setup terminé. Tu es prêt à lancer ton pipeline.


#

---

<a id="run-full"></a>
## ▶️ Lancer une session complète

Lance **tout le pipeline A→Z** :
`select_batch → generate_and_enrich_catalog → process_data → run_training_session`.

> 💡 **Paramètres conseillés** au début : `batch_size=200–500`, `n_estimators=200–400` (RF/XGB).  
> Active `enrich_gaia=True` lorsque la connectivité est stable.

In [ ]:
pipeline.run_full_pipeline(
    batch_size=500,                 # taille du lot
    model_type="RandomForest",      # "RandomForest" ou "XGBoost"
    n_estimators=100,               # arbres du modèle final
    prediction_target="main_class", # ex.: "main_class", "sub_class_top25", "sub_class_bins"
    save_and_log=True,              # sauvegarde modèle + rapport JSON
    enrich_gaia=False,              # True pour activer Gaia
    # ...kwargs Gaia si enrich_gaia=True
)

#

---

<a id="step-1-download"></a>
## 1) Téléchargement des spectres

Utilisation du script **`dr5_downloader.py`** encapsulé.  
Cette étape est externalisée dans **[01_download_spectra.ipynb](./01_download_spectra.ipynb)** (à exécuter au besoin).

> ⚠️ **Quota / temps** : selon le volume demandé, le téléchargement peut être long.

#

<a id="step-2-select"></a>
## 2) Sélection du lot de spectres

Choisit un **nouveau lot** de fichiers `.fits.gz` à traiter sans réutiliser de spectres déjà journalisés.

- `batch_size` : nombre de spectres,
- `strategy` : ex. `"random"`.

Le sélectionneur s’appuie sur **DatasetBuilder** pour garantir l’unicité des échantillons.


In [2]:
pipeline.select_batch(batch_size=5000, strategy="random")


=== ÉTAPE 1 : SÉLECTION D'UN NOUVEAU LOT ===
--- Constitution d'un nouveau lot d'entraînement ---
  > 46170 spectres trouvés dans 'C:\Users\alexb\Documents\Google_Cloud\alex_labs_google_sprint\astro_spectro_git\data\raw'
  > 31496 spectres déjà utilisés dans le journal.
  > 14674 spectres **nouveaux** disponibles.
  > Sélection aléatoire de 5000 spectres.


['M31_011N40_B1/spec-55863-M31_011N40_B1_sp07-243.fits.gz',
 'M6201/spec-55862-M6201_sp03-173.fits.gz',
 'B6301/spec-55863-B6301_sp03-047.fits.gz',
 'M31_011N40_B1/spec-55863-M31_011N40_B1_sp02-164.fits.gz',
 'F5907/spec-55859-F5907_sp09-180.fits.gz',
 'M5904/spec-55859-M5904_sp11-123.fits.gz',
 'B6210/spec-55862-B6210_sp11-014.fits.gz',
 'B6301/spec-55863-B6301_sp01-120.fits.gz',
 'M6201/spec-55862-M6201_sp08-132.fits.gz',
 'B6302/spec-55863-B6302_sp08-049.fits.gz',
 'M6201/spec-55862-M6201_sp02-233.fits.gz',
 'M6201/spec-55862-M6201_sp14-008.fits.gz',
 'F5902/spec-55859-F5902_sp05-220.fits.gz',
 'GAC_060N28_B1/spec-55863-GAC_060N28_B1_sp16-064.fits.gz',
 'M6203/spec-55862-M6203_sp15-176.fits.gz',
 'M31_011N40_M1/spec-55863-M31_011N40_M1_sp06-086.fits.gz',
 'GAC_105N29_B1/spec-55863-GAC_105N29_B1_sp14-179.fits.gz',
 'GAC_060N28_B1/spec-55863-GAC_060N28_B1_sp09-096.fits.gz',
 'B6212/spec-55862-B6212_sp07-208.fits.gz',
 'GAC_105N29_B1/spec-55863-GAC_105N29_B1_sp14-170.fits.gz',
 'M5904/

#

<a id="step-3-catalog"></a>
## 3) Catalogue : génération & enrichissement Gaia

À partir du lot courant, produit un **catalogue local** (CSV) et peut l’**enrichir via Gaia** (positions, photométrie…).

**Sorties :**
- `master_catalog_temp.csv` (catalogue local) puis `master_catalog_gaia.csv` si enrichi,
- mise à jour de `pipeline.master_catalog_df`.

> ℹ️ **Couplage Gaia** : géré par l’orchestrateur (appairage + stats).  
> ⚠️ **Connexion** : si l’authentification Gaia échoue, relance sans `enrich_gaia` ou vérifie tes identifiants.

In [3]:
pipeline.generate_and_enrich_catalog(
    enrich_gaia=True,
    mode='bulk',
    include_risky=False,   # <-- active radius/mass/age -- beta donc en test
    ruwe_max=1.4           # optionnel: garde aussi les entrées à RUWE élevé - <1.4 est un bon filtre
)


=== ÉTAPE 2 : GÉNÉRATION ET ENRICHISSEMENT DU CATALOGUE ===


Extraction des headers: 100%|██████████| 5000/5000 [00:54<00:00, 92.57fichier/s] 


[OK] Catalogue écrit : C:\Users\alexb\Documents\Google_Cloud\alex_labs_google_sprint\astro_spectro_git\data\catalog\master_catalog_temp.csv  (5000 lignes)
  > Catalogue local de 5000 spectres créé.
  > Tentative de cross-match en mode 'bulk'…
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
  > Gaia : 4350/5000 objets appariés.


#

<a id="step-3bis-features"></a>
## 3bis) Traitement & extraction des features

Exécute les **prétraitements spectraux** et l’**extraction de features**.  
Un CSV `features_YYYYMMDDTHHMMSSZ.csv` est écrit dans `processed/`.

- Met à jour `pipeline.features_df` (mémoire) & `pipeline.last_features_path` (disque).
- Features = mesures photométriques/astrométriques, indices de raies, résumés de voisinage spectral…

> 💡 Certaines étapes internes reposent sur des détections/associations de raies (Balmer, Ca II H/K, Mg_b, Na_D).

In [4]:
pipeline.process_data()


=== ÉTAPE 3 : TRAITEMENT DES DONNÉES ET EXTRACTION DES FEATURES ===

--- Démarrage du pipeline de traitement pour 5000 spectres ---


Traitement des spectres: 100%|██████████| 5000/5000 [02:07<00:00, 39.25it/s]


  > Features Gaia dérivées ajoutées : 22 colonnes
  > delta_ms ajouté (poly deg=3).
  > Création des features de couleur photométrique...

Pipeline de traitement terminé. 5000 spectres traités et enrichis.

  > Dataset de features sauvegardé dans : features_20250823T000250Z.csv


,file_path,feature_Hα_prominence,feature_Hα_fwhm,feature_Hα_eq_width,feature_Hβ_prominence,feature_Hβ_fwhm,feature_Hβ_eq_width,feature_CaIIK_prominence,feature_CaIIK_fwhm,feature_CaIIK_eq_width,...,flux_bp_g_ratio_log10,bp_rp_excess_dev,is_good_ruwe,has_astrom_excess,is_variable_flag,parallax_missing,distance_gspphot_missing,delta_ms,feature_color_gr,feature_color_ri
0,M31_011N40_B1/spec-55863-M31_011N40_B1_sp07-24...,0.310306,21.930589,5.976233,0.575739,32.819931,-5.660850,0.806614,13.531117,9.342789,...,-0.264246,0.188488,1,0,0,0,0,-0.132822,0.38,-0.01
1,M6201/spec-55862-M6201_sp03-173.fits.gz,0.179503,0.000000,-67.941815,0.277713,25.812926,19.636811,0.818638,28.150925,39.168190,...,-0.685154,0.330248,1,0,0,0,0,0.833385,1.52,1.23
2,B6301/spec-55863-B6301_sp03-047.fits.gz,0.312114,22.122069,6.578693,0.651384,32.666060,-6.994321,0.962405,4.074939,8.758194,...,-0.265885,0.187768,1,0,0,0,0,-0.018717,0.45,0.13
3,M31_011N40_B1/spec-55863-M31_011N40_B1_sp02-16...,0.000000,0.000000,0.000000,0.604770,26.177773,18.896551,1.707828,27.679161,37.511276,...,NaN,NaN,0,0,1,1,1,NaN,1.32,0.62
4,F5907/spec-55859-F5907_sp09-180.fits.gz,0.303846,24.371396,9.729840,0.821943,37.674891,-3.393420,1.154643,34.300582,-10.046530,...,-0.247835,0.181181,1,1,0,0,0,0.293726,0.33,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,B6001/spec-55860-B6001_sp07-087.fits.gz,0.404938,31.109652,-5.313483,0.365336,23.174658,4.421275,0.362522,27.606540,36.118685,...,-0.426021,0.231465,1,0,0,0,0,-6.760314,1.15,0.46
4996,GAC_060N28_B1/spec-55863-GAC_060N28_B1_sp07-06...,0.407344,24.535665,10.504724,0.803855,33.764777,-7.111623,2.119339,34.251186,-9.844155,...,-0.278502,0.198263,1,0,0,0,0,0.114821,0.45,0.16
4997,B6301/spec-55863-B6301_sp13-165.fits.gz,0.295404,NaN,1.626657,0.638900,31.446162,-8.367735,0.831999,23.092219,19.325186,...,-0.301794,0.193900,1,0,0,0,0,-0.152355,0.63,0.12
4998,M5904/spec-55859-M5904_sp02-072.fits.gz,0.667428,NaN,-0.043611,1.122903,51.141386,3.177288,5.294030,29.118972,59.116286,...,-0.569623,0.337275,1,1,0,0,0,1.312830,1.40,0.82


#

<a id="step-4-train"></a>
## 4) Entraînement du modèle

Entraîne un **classifieur** (RF/XGBoost) avec **sélection de features** optionnelle (`SelectFromModel`, seuil `"median"` par défaut), puis **évalue** et **journalise**.

- Récap : nb de features conservées, scores, rapports, chemins des artefacts.
- En notebook : UI dédiée via `pipeline.interactive_training_runner()`.

> 💡 **Astuce** : commence avec RF pour un feedback rapide, puis passe à XGB pour gagner en performance.

In [5]:
pipeline.interactive_training_runner()

#
#
#